<a href="https://colab.research.google.com/github/u-k-d/u-k-d/blob/main/api_recomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  from google.colab import drive
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gradio pandas langchain_huggingface langchain_chroma

In [3]:
import gradio as gr
import json, string
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
import chromadb.utils.embedding_functions as embedding_functions

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,

)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
with open('/content/drive/MyDrive/recomender/upsell.JSON',encoding="utf-8-sig") as file:
    data = json.load(file)
    df = pd.DataFrame(data)
    df = df.reset_index()


In [5]:
CHROMADB_DIR_PATH = "/content/drive/MyDrive/recomender/chroma/upsell_embedding/"  # Change it to the path on your directory

In [6]:
db_products = Chroma(embedding_function=hf,persist_directory=CHROMADB_DIR_PATH) # Loading Chromadb vector embeddings


In [7]:
def preprocess_text(text):
    text = str(text)
    PUNCT_TO_REMOVE = string.punctuation
    ans = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    ans.replace("\n","")
    ans = ans.lower()
    return ans

In [8]:
def retrive_recomended_products(
    query: str,
    initial_top_k: int = 5,
    final_top_k: int = 5,
) -> pd.DataFrame:
    recs = docs = db_products.similarity_search(query, k = initial_top_k)
    prod_list = [str(rec.page_content.strip('"').split()[0]) for rec in recs]

    _df = df[df["Hoofdartikel"].isin(prod_list)].head(final_top_k)
    prodcut_recomendation = ()
    return _df
    # for x in _df["Combinaties"]:
    #     for y in x:
    #         if y["Prodid"] not in _tmp.keys():
    #             _tmp.append(y["Prodid"])
    #             _tmp[y["Prodid"]] = 0
    #         else:
    #             _tmp[y["Prodid"]] += 1
    #             # _tmp.append(y["Prodid"])
    #             # print(F" ProdId : {y["Prodid"] } Title : {y["Omschrijving"] }")



In [9]:
def recomended_products(query:str):
    query = preprocess_text(str(query))
    if(len(query) == 0 ):
        return([("Invalid Key", "This product id is not in the system")])
    # query = query["products_description"].iloc[0]
    # query  = query.replace("\n", "")
    recomendations = retrive_recomended_products(query)
    # print(recomendations)
    results=  []
    _tmp = []
    for _,x in recomendations.iterrows():
        for y in x["Combinaties"]:
            if y["Prodid"] not in _tmp:
                _tmp.append(y["Prodid"])
                results.append((x["Hoofdartikel"],x["Titel"],y["Prodid"],y["Omschrijving"]))
                # _tmp[y["Prodid"]] = 0
            else:
                pass
                # _tmp[y["Prodid"]] += 1
                # _tmp.append(y["Prodid"])
                # print(F" ProdId : {y["Prodid"] } Title : {y["Omschrijving"] }")
    return results


In [10]:
query = """CONTINENTAL Tire Double Fighter III rolling easily on asphalt, offroad uses the highly profiled shoulder. combines high speed on the road and safe grip in the terrain, it's specially for MTB's which riding on the street 50-559, 26 x 1,9, black"""


In [11]:
retrive_recomended_products(query)

,index,Hoofdartikel,Titel,Informatie,Combinaties
834,834,142466,Continental Giro Buitenband Tubular 22-622 - Z...,Deze betrouwbare tubular favoriet met het beke...,"[{'Prodid': '101349', 'Omschrijving': 'Schwalb..."
12333,12333,693759,"Michelin DH34 MagiX Band 27.5 x 2.40\"" TL-R - ...","Michelin DH 34 27,5\"" draadband is de snelste ...","[{'Prodid': '624655', 'Omschrijving': 'Vittori..."
12814,12814,713861,"Pirelli Cinturato Gravel 28 x 1.35\"" Mixed - Z...",Inch Maat = 28 x 1.35 ETRTO Maat = 35-622 ...,"[{'Prodid': '210020', 'Omschrijving': 'Micheli..."
12815,12815,713862,"Pirelli Cinturato Gravel 28 x 1.50\"" Mixed - Z...",Inch Maat = 28 x 1.50 ETRTO Maat = 40-622 ...,"[{'Prodid': '101359', 'Omschrijving': 'Schwalb..."
12819,12819,713866,"Pirelli Cinturato Gravel 28 x 1.35\"" Hard - Zw...",Inch Maat = 28 x 1.35 ETRTO Maat = 35-622 ...,"[{'Prodid': '210020', 'Omschrijving': 'Micheli..."


In [12]:
with gr.Blocks(theme = gr.themes.Glass()) as dashboard:
    gr.Markdown("# Product Recomendations")

    with gr.Row():
        user_query = gr.Textbox(label = "Please enter product description", placeholder= "101385")
        submit_button = gr.Button("Find Recomendations")

    gr.Markdown("# Recomendations")
    output = gr.Dataframe(label = "Recomended Items", headers=["Matched Object","ProdId","Description"])

    submit_button.click(fn = recomended_products, inputs= [user_query], outputs = output)



if __name__ == "__main__":
    dashboard.launch()




It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://32418fba301b890289.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
